In [ ]:
# ==============================================================================
# CONTACT SIMULATION AUTOMATION SCRIPT
# ==============================================================================
# Description: Automates the setup, solution, and result export for a contact 
#              simulation between grippers and a cylinder using Ansys Mechanical.
# Author: [Your Name]
# Date: [Current Date]
# ==============================================================================

import os
import time
import datetime
import math
import csv
from pathlib import Path

# --- Ansys Mechanical Dependencies ---
# Note: These imports are required when running inside the Ansys IronPython console.
from ansys.mechanical.core import App
from Ansys.Mechanical.DataModel.Enums import *
from Ansys.ACT.Mechanical.Utilities import GeometryImportPreferences

# --- Visualization Dependencies ---
from matplotlib import image as mpimg
from matplotlib import pyplot as plt

# ==============================================================================
# 1. SIMULATION PARAMETERS (CONSTANTS)
# ==============================================================================
# Geometry Dimensions
CYLINDER_RADIUS = 12.0
HOLE_RADIUS = 2.0 

# Geometry Import Indices (0-based index of bodies in the tree)
IDX_CYLINDER = 0
IDX_GRIPPER_1 = 1
IDX_GRIPPER_2 = 2

# Named Selection (NS) Tags
NS_NAME_CYLINDER_FACE = "NS_Cylinder_Flat_Face"
NS_NAME_GRIPPER_1_HOLE = "NS_Hole_Gripper1"
NS_NAME_GRIPPER_2_HOLE = "NS_Hole_Gripper2"
NS_NAME_CONTACT_G1 = "NS_Contact_Gripper1_Inner"
NS_NAME_CONTACT_G2 = "NS_Contact_Gripper2_Inner"
NS_NAME_CONTACT_CYL_FACE = "NS_Contact_Cylinder_Face"

# Initialize App
app = App()
app.update_globals(globals())
print(f"Connected to: {app}")

# Start Timer
start_time = time.time()

In [ ]:
def display_image(image_path):
    """
    Displays an image within the Jupyter Notebook environment.
    """
    try:
        plt.figure(figsize=(16, 9))
        plt.imshow(mpimg.imread(image_path))
        plt.xticks([])
        plt.yticks([])
        plt.axis("off")
        plt.show()
        plt.close() # Close figure to save memory
    except Exception as e:
        print(f"Warning: Could not display image {image_path}. Error: {e}")

def create_hole_named_selection(body_index, ns_name):
    """
    Identifies a cylindrical hole face in a specific body based on the global 
    HOLE_RADIUS parameter and creates a Named Selection.
    
    Args:
        body_index (int): Index of the body in the geometry tree.
        ns_name (str): Name for the new Named Selection.
    """
    all_bodies = Model.GetChildren(DataModelObjectCategory.Body, True)

    if len(all_bodies) > body_index:
        body = all_bodies[body_index]
        geo_body = body.GetGeoBody()
        found_faces = []

        for face in geo_body.Faces:
            try:
                # Check if face radius matches target with a small tolerance
                if math.isclose(face.Radius, HOLE_RADIUS, rel_tol=1e-5):
                    found_faces.append(face)
            except:
                pass # Not a cylindrical face

        if not found_faces:
            print(f"WARNING: No face found with radius {HOLE_RADIUS} mm in body '{body.Name}'.")
            return

        # Create Named Selection
        ns = Model.AddNamedSelection()
        ns.Name = ns_name
        selection = ExtAPI.SelectionManager.CreateSelectionInfo(SelectionTypeEnum.GeometryEntities)
        selection.Ids = [f.Id for f in found_faces]
        ns.Location = selection
        print(f"-> Named Selection '{ns_name}' created successfully.")
    else:
        print(f"ERROR: Body index {body_index} out of range. Total bodies: {len(all_bodies)}")

def create_ns_by_proximity(seeker_body_idx, target_body_idx, ns_name):
    """
    Finds the face on the 'seeker' body that is physically closest to the 
    centroid of the 'target' body. Useful for identifying contact faces.
    """
    all_bodies = Model.GetChildren(DataModelObjectCategory.Body, True)

    if len(all_bodies) <= seeker_body_idx or len(all_bodies) <= target_body_idx:
        print("ERROR: Body indices out of range.")
        return

    seeker_body = all_bodies[seeker_body_idx]
    target_body = all_bodies[target_body_idx]

    # 1. Get Target Centroid
    target_centroid = target_body.GetGeoBody().Centroid

    # 2. Iterate faces to find minimum distance
    geo_body_seeker = seeker_body.GetGeoBody()
    closest_face = None
    min_distance = float('inf')

    for face in geo_body_seeker.Faces:
        face_centroid = face.Centroid
        # Euclidean distance calculation
        dist = math.sqrt(
            (face_centroid[0] - target_centroid[0])**2 +
            (face_centroid[1] - target_centroid[1])**2 +
            (face_centroid[2] - target_centroid[2])**2
        )

        if dist < min_distance:
            min_distance = dist
            closest_face = face

    if closest_face is None:
        print(f"WARNING: No candidate face found on '{seeker_body.Name}'.")
        return

    # 3. Create Named Selection
    ns = Model.AddNamedSelection()
    ns.Name = ns_name
    selection = ExtAPI.SelectionManager.CreateSelectionInfo(SelectionTypeEnum.GeometryEntities)
    selection.Ids = [closest_face.Id]
    ns.Location = selection

def create_ns_cylindrical_face(body_index, ns_name, search_radius):
    """
    Finds faces matching a specific radius on a target body.
    """
    all_bodies = Model.GetChildren(DataModelObjectCategory.Body, True)
    if len(all_bodies) > body_index:
        body = all_bodies[body_index]
        found_faces = [f for f in body.GetGeoBody().Faces if hasattr(f, 'Radius') and math.isclose(f.Radius, search_radius, rel_tol=1e-5)]
        
        if not found_faces:
            print(f"WARNING: No face found with radius {search_radius} mm.")
            return

        ns = Model.AddNamedSelection()
        ns.Name = ns_name
        selection = ExtAPI.SelectionManager.CreateSelectionInfo(SelectionTypeEnum.GeometryEntities)
        selection.Ids = [f.Id for f in found_faces]
        ns.Location = selection

def apply_fixed_support(ns_name):
    """Applies a Fixed Support boundary condition to a Named Selection."""
    ns_object = next((ns for ns in Model.NamedSelections.Children if ns.Name == ns_name), None)
    
    if ns_object is None:
        print(f"ERROR: Named Selection '{ns_name}' not found.")
        return

    analysis = Model.Analyses[0]
    fixed_support = analysis.AddFixedSupport()
    selection = ExtAPI.SelectionManager.CreateSelectionInfo(SelectionTypeEnum.GeometryEntities)
    selection.Ids = ns_object.Location.Ids
    fixed_support.Location = selection
    fixed_support.Suppressed = False

def apply_moment_load(ns_name, magnitude, direction_vector):
    """
    Applies a Moment load to a specific Named Selection.
    
    Args:
        ns_name (str): Target Named Selection.
        magnitude (float): Moment magnitude in N*m.
        direction_vector (list): [x, y, z] vector components.
    """
    ns_object = next((ns for ns in Model.NamedSelections.Children if ns.Name == ns_name), None)
    
    if ns_object is None:
        print(f"ERROR: Named Selection '{ns_name}' not found.")
        return

    analysis = Model.Analyses[0]
    moment_load = analysis.AddMoment()
    
    # Location
    selection = ExtAPI.SelectionManager.CreateSelectionInfo(SelectionTypeEnum.GeometryEntities)
    selection.Ids = ns_object.Location.Ids
    moment_load.Location = selection
    
    # Definition
    moment_load.DefineBy = LoadDefineBy.Vector
    moment_load.Magnitude.Output.SetDiscreteValue(0, Quantity(magnitude, "N m"))
    
    # Direction
    vector = Ansys.ACT.Math.Vector3D(direction_vector[0], direction_vector[1], direction_vector[2])
    moment_load.Direction = vector
    moment_load.Suppressed = False

def progress_bar(current_step, total_steps, message=""):
    """Prints a simple text-based progress bar."""
    percent = int((current_step / total_steps) * 100)
    bar = "█" * (percent // 5) + "_" * (20 - percent // 5)
    print(f"[{bar}] {percent}%  -> {message}")

In [ ]:
def generate_and_export_simulation_views(output_folder, export_settings, display_func):
    """
    Generates and exports key post-processing images to the output folder.
    
    Args:
        output_folder (str): Directory path to save images.
        export_settings (object): Ansys GraphicsImageExportSettings object.
        display_func (func): Callback function to display image in notebook.
    """
    print("\n--- STARTING VISUALIZATION EXPORT ---")
    
    # Ensure Analysis Context
    try:
        analysis = Model.Analyses[0]
        solution = analysis.Solution
        all_bodies = Model.GetChildren(DataModelObjectCategory.Body, True)
        all_ns = Model.NamedSelections.Children
    except Exception as e:
        print(f"Error initializing visualization context: {e}")
        return

    # --- Helper: Isolate and Highlight ---
    def isolate_and_capture(ns_names, filename, rot_y, rot_x=-30):
        """Isolates specific Named Selections and captures a screenshot."""
        target_bodies = []
        ns_to_activate = []

        try:
            # Identify bodies associated with the Named Selections
            for name in ns_names:
                ns_obj = next((ns for ns in all_ns if ns.Name == name), None)
                if ns_obj:
                    ns_to_activate.append(ns_obj)
                    if ns_obj.Location.Entities:
                        body_id = ns_obj.Location.Entities[0].Body.Id
                        body = next((b for b in all_bodies if b.GetGeoBody().Id == body_id), None)
                        if body and body not in target_bodies:
                            target_bodies.append(body)
            
            # Visibility Logic
            if not target_bodies: target_bodies = all_bodies
            for b in all_bodies: b.Visible = False
            for b in target_bodies: b.Visible = True

            if ns_to_activate: Tree.Activate(ns_to_activate)
            
            # Camera Setup
            Graphics.Camera.SetFit()
            Graphics.Camera.SetSpecificViewOrientation(ViewOrientationType.Front)
            Graphics.Camera.Rotate(-90, CameraAxisType.ScreenX)
            Graphics.Camera.Rotate(180, CameraAxisType.ScreenY)
            Graphics.Camera.Rotate(rot_y, CameraAxisType.ScreenY)
            Graphics.Camera.Rotate(45, CameraAxisType.ScreenY)
            Graphics.Camera.Rotate(30, CameraAxisType.ScreenX)

            # Export
            filepath = os.path.join(output_folder, filename)
            Graphics.ExportImage(filepath, GraphicsImageExportFormat.PNG, export_settings)
            print(f"Exported: {filename}")
            display_func(filepath)

        except Exception as e:
            print(f"Error exporting {filename}: {e}")
        finally:
            # Restore state
            for b in all_bodies: b.Visible = True
            Tree.Activate([Model.Geometry])

    # --- VIEW 1: Holes (Combined) ---
    isolate_and_capture(
        [NS_NAME_GRIPPER_1_HOLE, NS_NAME_GRIPPER_2_HOLE],
        "View_Holes_Combined.png", rot_y=0
    )

    # --- VIEW 2: Contact Surfaces ---
    isolate_and_capture(
        [NS_NAME_CONTACT_G1, NS_NAME_CONTACT_G2, NS_NAME_CONTACT_CYL_FACE],
        "View_Contacts_Combined.png", rot_y=0
    )

    # --- VIEW 3: Results (Total Deformation) ---
    try:
        def_result = solution.Children[1] # Assuming Index 1 is Total Def
        def_result.EvaluateAllResults()
        Tree.Activate([def_result])
        
        Graphics.Camera.SetFit()
        Graphics.Camera.SetSpecificViewOrientation(ViewOrientationType.Front)
        Graphics.Camera.Rotate(-90, CameraAxisType.ScreenX)
        Graphics.Camera.Rotate(180, CameraAxisType.ScreenY)
        Graphics.Camera.Rotate(45, CameraAxisType.ScreenY)
        Graphics.Camera.Rotate(30, CameraAxisType.ScreenX)
        
        filepath = os.path.join(output_folder, "Result_Total_Deformation.png")
        Graphics.ExportImage(filepath, GraphicsImageExportFormat.PNG, export_settings)
        print("Exported: Result_Total_Deformation.png")
        display_func(filepath)
    except Exception as e:
        print(f"Error exporting Deformation result: {e}")

    # Restore Selection
    Tree.Activate([Model.Geometry])
    print("--- VISUALIZATION COMPLETED ---")

In [ ]:
# --- Graphics Settings Configuration ---
image_export_format = GraphicsImageExportFormat.PNG
settings_720p = Ansys.Mechanical.Graphics.GraphicsImageExportSettings()
settings_720p.Resolution = GraphicsResolutionType.EnhancedResolution
settings_720p.Background = GraphicsBackgroundType.White
settings_720p.Width = 1280
settings_720p.Height = 720
settings_720p.CurrentGraphicsDisplay = False

def run_simulation_workflow(cad_file_path, base_target_folder):
    """
    Executes the full simulation pipeline for a single CAD file.
    
    1. Imports Geometry
    2. Sets up Named Selections, Joints, and Contacts
    3. Meshes and Solves
    4. Exports Data and Images
    
    Returns:
        dict: A dictionary containing the simulation results (forces, pressures).
    """
    
    # Initialize Progress Tracking
    TOTAL_STEPS = 11
    step = 0
    file_name = os.path.basename(cad_file_path)
    print(f"\n{'='*60}")
    print(f"| 🚀 STARTING SIMULATION: {file_name}")
    print(f"{'='*60}")

    # 1. New Project & Import
    app.new()
    app.update_globals(globals())
    
    step += 1
    progress_bar(step, TOTAL_STEPS, "Importing Geometry")
    
    geom_import = Model.GeometryImportGroup.AddGeometryImport()
    import_prefs = GeometryImportPreferences()
    import_prefs.ProcessNamedSelections = True
    import_prefs.ProcessCoordinateSystems = True
    geom_import.Import(cad_file_path, GeometryImportPreference.Format.Automatic, import_prefs)
    
    ExtAPI.Application.ActiveUnitSystem = MechanicalUnitSystem.StandardMKS

    # 2. Named Selections Generation
    step += 1
    progress_bar(step, TOTAL_STEPS, "Generating Named Selections")
    
    # Define Holes (Joint Locations)
    create_hole_named_selection(IDX_GRIPPER_1, NS_NAME_GRIPPER_1_HOLE)
    create_hole_named_selection(IDX_GRIPPER_2, NS_NAME_GRIPPER_2_HOLE)
    
    # Define Contacts (Proximity)
    create_ns_by_proximity(IDX_GRIPPER_1, IDX_CYLINDER, NS_NAME_CONTACT_G1)
    create_ns_by_proximity(IDX_GRIPPER_2, IDX_CYLINDER, NS_NAME_CONTACT_G2)
    
    # Define Cylinder Face
    create_ns_cylindrical_face(IDX_CYLINDER, NS_NAME_CONTACT_CYL_FACE, CYLINDER_RADIUS)

    # 3. Joint Setup
    step += 1
    progress_bar(step, TOTAL_STEPS, "Configuring Revolute Joints")
    
    connections = Model.Connections
    joint_group = connections.AddConnectionGroup()
    joint_group.Name = "Joints_Revolute_Axes"
    
    for i, ns_name in enumerate([NS_NAME_GRIPPER_1_HOLE, NS_NAME_GRIPPER_2_HOLE]):
        ns_obj = next((ns for ns in Model.NamedSelections.Children if ns.Name == ns_name), None)
        if ns_obj:
            joint = joint_group.AddJoint()
            joint.ConnectionType = JointScopingType.BodyToGround
            joint.Type = JointType.Revolute
            joint.MobileLocation = ns_obj
            # Set Behavior to Deformable to avoid stress singularities
            joint.ReferenceBehavior = LoadBehavior.Deformable
            joint.MobileBehavior = LoadBehavior.Deformable
            joint.CoordinateSystem = Model.CoordinateSystems.Children[0] # Global CS
            joint.PrincipalAxisZ = 1
            if i == 1: joint.MobileRelaxationMethod = True # Relax second joint

    # 4. Contact Setup
    step += 1
    progress_bar(step, TOTAL_STEPS, "Configuring Contacts")
    
    contact_group = connections.AddConnectionGroup()
    contact_group.Name = "Contacts_Gripper_Cylinder"
    
    ns_cyl_contact = next((ns for ns in Model.NamedSelections.Children if ns.Name == NS_NAME_CONTACT_CYL_FACE), None)
    
    for ns_name in [NS_NAME_CONTACT_G1, NS_NAME_CONTACT_G2]:
        ns_src = next((ns for ns in Model.NamedSelections.Children if ns.Name == ns_name), None)
        if ns_src and ns_cyl_contact:
            contact = contact_group.AddContactRegion()
            contact.SourceLocation = ns_src.Location
            contact.TargetLocation = ns_cyl_contact.Location
            contact.ContactType = ContactType.Frictionless

    # 5. Meshing
    step += 2 # Skipped tool step in counter for brevity
    progress_bar(step, TOTAL_STEPS, "Generating Mesh")
    
    Model.AddTransientStructuralAnalysis()
    analysis = Model.Analyses[0]
    
    mesh = Model.Mesh
    mesh.ElementOrder = ElementOrder.Linear
    mesh.ElementSize = Quantity("2 [mm]")
    
    # Local Refinement on Contacts
    refinement = mesh.AddRefinement()
    refinement.Name = "Refinement_Contact_Faces"
    refinement.NumberOfRefinements = 2
    
    ids_to_refine = []
    for ns_name in [NS_NAME_CONTACT_G1, NS_NAME_CONTACT_G2]:
        ns = next((ns for ns in Model.NamedSelections.Children if ns.Name == ns_name), None)
        if ns: ids_to_refine.extend(list(ns.Location.Ids))
            
    sel_loc = ExtAPI.SelectionManager.CreateSelectionInfo(SelectionTypeEnum.GeometryEntities)
    sel_loc.Ids = ids_to_refine
    refinement.Location = sel_loc
    
    mesh.GenerateMesh()

    # 6. Analysis Settings
    step += 1
    progress_bar(step, TOTAL_STEPS, "Analysis Settings")
    
    settings = analysis.AnalysisSettings
    settings.ProcessorCount = 4
    settings.NumberOfSteps = 1
    settings.StepEndTime = Quantity("1 [s]")
    settings.LargeDeflection = True
    settings.AutomaticTimeStepping = AutomaticTimeStepping.On
    settings.InitialTimeStep = Quantity("0.01 [s]")
    settings.MinimumTimeStep = Quantity("0.01 [s]")
    settings.MaximumTimeStep = Quantity("0.01 [s]")
    settings.NodalForces = OutputControlsNodalForcesType.Yes

    # 7. Boundary Conditions (Fixed Support & Moments)
    step += 1
    progress_bar(step, TOTAL_STEPS, "Applying Loads")
    
    # Find flat face on cylinder (radius = 0)
    cyl_body = Model.GetChildren(DataModelObjectCategory.Body, True)[IDX_CYLINDER]
    flat_faces = [f for f in cyl_body.GetGeoBody().Faces if math.isclose(f.Radius, 0.0, abs_tol=1e-9)]
    
    if flat_faces:
        ns_fix = Model.AddNamedSelection()
        ns_fix.Name = NS_NAME_CYLINDER_FACE
        sel = ExtAPI.SelectionManager.CreateSelectionInfo(SelectionTypeEnum.GeometryEntities)
        sel.Ids = [flat_faces[0].Id]
        ns_fix.Location = sel
        apply_fixed_support(NS_NAME_CYLINDER_FACE)
    
    # Apply Moments (Opposing directions)
    apply_moment_load(NS_NAME_GRIPPER_1_HOLE, 1, [0, 0, 1])
    apply_moment_load(NS_NAME_GRIPPER_2_HOLE, -1, [0, 0, 1])

    # 8. Result Setup (Probes)
    step += 1
    progress_bar(step, TOTAL_STEPS, "Configuring Result Probes")
    
    solution = analysis.Solution
    solution.AddTotalDeformation()
    contact_tool = solution.AddContactTool()
    contact_tool.AddPressure()
    
    # Force Probes on Contacts
    for i, contact_reg in enumerate(contact_group.Children):
        probe = solution.AddForceReaction()
        probe.Name = f"Force_Reaction_Gripper_{i+1}"
        probe.LocationMethod = LocationDefinitionMethod.ContactRegion
        probe.ContactRegionSelection = contact_reg

    # 9. Solve
    step += 1
    progress_bar(step, TOTAL_STEPS, "Solving (This may take time)...")
    analysis.Solve()

    # 10. Output Generation
    step += 1
    progress_bar(step, TOTAL_STEPS, "Exporting Results")
    
    # Setup Output Directory
    file_base = os.path.splitext(file_name)[0]
    output_folder = os.path.join(base_target_folder, file_base)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        
    # Generate Images
    generate_and_export_simulation_views(output_folder, settings_720p, display_image)
    
    # Extract Numeric Data
    probe1 = solution.Children[3] # Gripper 1 Force
    probe2 = solution.Children[4] # Gripper 2 Force
    pressure = solution.Children[2].Children[1] # Contact Pressure
    
    probe1.EvaluateAllResults()
    probe2.EvaluateAllResults()
    pressure.EvaluateAllResults()
    
    results_data = {
        'FileName': file_base,
        'Fx1': probe1.XAxis.Value, 'Fy1': probe1.YAxis.Value, 'Ftot1': probe1.Total.Value,
        'Fx2': probe2.XAxis.Value, 'Fy2': probe2.YAxis.Value, 'Ftot2': probe2.Total.Value,
        'Pressure_Max': pressure.Maximum.Value,
        'Pressure_Avg': pressure.Average.Value
    }
    
    # Save TXT Summary
    txt_path = os.path.join(output_folder, f"{file_base}_SUMMARY.txt")
    with open(txt_path, "w", encoding="utf-8") as f:
        f.write(f"SIMULATION SUMMARY: {file_name}\n")
        f.write(f"Date: {datetime.datetime.now()}\n\n")
        for k, v in results_data.items():
            f.write(f"{k}: {v}\n")
    print(f"-> Saved Summary: {txt_path}")
    
    # Save Project (.mechdb)
    mech_path = os.path.join(base_target_folder, f"{file_base}.mechdb")
    app.save(mech_path)
    print(f"-> Project Saved: {mech_path}")
    
    step += 1
    progress_bar(step, TOTAL_STEPS, "Completed")
    
    return results_data

In [ ]:
def run_batch_simulation(directory):
    """
    Iterates through all STEP files in a directory and runs the simulation workflow.
    Exports a Master CSV with all results.
    """
    directory = Path(directory)
    # Filter for CAD files
    cad_files = [str(p) for p in directory.iterdir() if p.suffix.lower() in ['.step', '.stp']]
    
    if not cad_files:
        print("No STEP files found in directory.")
        return

    print(f"Found {len(cad_files)} files to process.")
    
    all_results = []
    
    for cad_file in cad_files:
        try:
            res = run_simulation_workflow(cad_file, str(directory))
            all_results.append(res)
        except Exception as e:
            print(f"CRITICAL ERROR simulating {cad_file}: {e}")
            
    # Export Master CSV
    if all_results:
        csv_path = directory / f"MASTER_RESULTS_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        keys = all_results[0].keys()
        with open(csv_path, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=keys)
            writer.writeheader()
            writer.writerows(all_results)
        print(f"\n✅ Batch Processing Complete. Master CSV saved: {csv_path}")

# --- EXECUTION ---
# UPDATE THIS PATH TO YOUR CAD FOLDER
INPUT_DIRECTORY = r"C:\prueba4" 

if os.path.exists(INPUT_DIRECTORY):
    run_batch_simulation(INPUT_DIRECTORY)
else:
    print(f"Error: Directory {INPUT_DIRECTORY} does not exist.")